# Word Embeddings in LevelDB

This example uses the [plyvel](https://plyvel.readthedocs.io/en/latest/) Connector within Python3 to store and retrieve various amounts of Word Embeddings.

In [ ]:
import plyvel
import subprocess
import shutil
import io
import time
import numpy
import plotly

# Dummy Embeddings

For testing purposes we will use randomly generated numpy arrays as dummy embbeddings.

In [ ]:
def embeddings(n=1000, dim=300):
    """
    Yield n tuples of random numpy arrays of *dim* length indexed by *n*
    """
    idx = 0
    while idx < n:
        yield (str(idx), numpy.random.rand(dim))
        idx += 1

# Conversion Functions

Since we can't just save a NumPy array into the database, we will convert it into a BLOB.

In [ ]:
def adapt_array(array):
    """
    Using the numpy.save function to save a binary version of the array,
    and BytesIO to catch the stream of data and convert it into a BLOB.
    """
    out = io.BytesIO()
    numpy.save(out, array)
    out.seek(0)

    return out.read()

def convert_array(blob):
    """
    Using BytesIO to convert the binary version of the array back into a numpy array.
    """
    out = io.BytesIO(blob)
    out.seek(0)

    return numpy.load(out)

In [ ]:
connection = plyvel.DB('./leveldb.embedding.db', create_if_missing=True)

In [ ]:
%%time
for key, emb in embeddings():
    arr = adapt_array(emb)
    connection.put(key.encode(), arr)

In [ ]:
%%time
for key, _ in embeddings():
    arr = connection.get(key.encode())
    emb = convert_array(arr)
    assert(type(emb) is numpy.ndarray)

In [ ]:
connection.close()

# Sample some data

To test the I/O we will write and read some data from the database. This may take a while.

In [ ]:
write_times = []
read_times = []
db_sizes = []
counts = [500, 1000, 5000, 10000, 50000, 100000]

for c in counts:
    shutil.rmtree('./leveldb.embedding.db', ignore_errors=True)
    connection = plyvel.DB('./leveldb.embedding.db', create_if_missing=True)
    
    start_time_write = time.time()
    for key, emb in embeddings(c):
        arr = adapt_array(emb)
        connection.put(key.encode(), arr)
    write_times.append(time.time() - start_time_write)
    
    start_time_read = time.time()
    for key, _ in embeddings(c):
        arr = connection.get(key.encode())
        emb = convert_array(arr)
        assert(type(emb) is numpy.ndarray)
    read_times.append(time.time() - start_time_read)

    connection.close()
    db_sized.append(subprocess.check_output(['du','-s', './leveldb.embedding.db']).split()[0].decode('utf-8'))
    
print('DONE')

# Results

In [ ]:
plotly.offline.init_notebook_mode(connected=True)
trace = plotly.graph_objs.Scatter(
    x = write_times,
    y = counts,
    mode = 'markers'
)
layout = plotly.graph_objs.Layout(title="LevelDB Write Times",
                xaxis=dict(title='Time in Seconds'),
                yaxis=dict(title='Embedding Count'))
data = [trace]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='jupyter-basic-scatter')

In [ ]:
plotly.offline.init_notebook_mode(connected=True)
trace = plotly.graph_objs.Scatter(
    x = read_times,
    y = counts,
    mode = 'markers'
)
layout = plotly.graph_objs.Layout(title="LevelDB Read Times",
                xaxis=dict(title='Time in Seconds'),
                yaxis=dict(title='Embedding Count'))
data = [trace]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='jupyter-basic-scatter')

In [ ]:
# DB Size
plotly.offline.init_notebook_mode(connected=True)
trace = plotly.graph_objs.Bar(
    x = counts,
    y = db_sizes
)
layout = plotly.graph_objs.Layout(title="Database Size",
                xaxis=dict(title='Size in KB'),
                yaxis=dict(title='Embedding Count'))
data = [trace]
fig = plotly.graph_objs.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='jupyter-basic_bar')